In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import zipfile as zf
filename
zf.ZipFile(filename).extractall('./')

In [ ]:
tr  = pd.read_csv('train.csv')
te  = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submit.csv')

### eda 

In [ ]:
tr.hist(figsize=(20,20))

In [ ]:
# !pip install pandas_profile
tr.profile_report()

### Preprocessing PipeLine 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class custom_ppr(BaseEstimator, TransformerMixin) :
    def __init__(self):
        pass
    def fit(self,x):
        return  self
    def transform(self,x):
        x = x.drop('index',axis = 1)

        return x

from sklearn.pipeline import Pipeline 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler 

num_pipe = Pipeline([
            ('custom',custom_ppr()),
            ('impute',SimpleImputer(strategy='median') ),
            ('scaler',StandardScaler())
])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

col_trans = ColumnTransformer([
            ('num',num_pipe,num_cols),
            ('cat',OneHotEncoder(),cat_cols)
])


In [ ]:
col_trans.fit(train)
trf_tr = col_trans.transform(train)
trf_cat = list(col_trans.transformers_[1][1].get_feature_names(cat_cols))
pd.DataFrame(trf_tr,columns = num_cols + trf_cat )

### model  

In [ ]:
# !pip install tune-sklearn
# !pip install scikit-optimize
from pycaret.classification import *
clf1 = setup(tr, target='target', session_id=123, log_experiment=True, experiment_name='exam_juice')
best_md = compare_models(n_select = 3)
tuned_adaboost = tune_model(best_md)
ensemble_model(tuned_form,method='Bagging')